In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import statsmodels.formula.api as smf


In [2]:
data = pd.read_excel('Chemitech.xlsx')
data.head()

,A,B,C
0,58,58,48
1,64,69,57
2,55,71,59
3,66,64,47
4,67,68,49


we will do anova first, and then solve the it using regression with a dummy variable

### ANOVA                      

In [3]:
data_r = pd.melt(data.reset_index(), id_vars=['index'], value_vars=['A','B', 'C'], var_name='treatments', value_name='value')
#data_r.columns = ['index', 'variable', 'value']
data_r.head(15)

,index,treatments,value
0,0,A,58
1,1,A,64
2,2,A,55
3,3,A,66
4,4,A,67
5,0,B,58
6,1,B,69
7,2,B,71
8,3,B,64
9,4,B,68


In [6]:
data_r.columns = ['index', 'treatments', 'value']
data_r.head()

,index,treatments,value
0,0,A,58
1,1,A,64
2,2,A,55
3,3,A,66
4,4,A,67


In [7]:
model = ols('value ~ C(treatments)', data = data_r).fit()
avova_table = sm.stats.anova_lm(model, type=1)
print(avova_table)

                 df  sum_sq     mean_sq         F    PR(>F)
C(treatments)   2.0   520.0  260.000000  9.176471  0.003818
Residual       12.0   340.0   28.333333       NaN       NaN


reject null hypothesis as F(0.0038) value is less than 0.05. so all means are not equal

regression analysis using dummy variables

In [13]:
just_dummies = pd.get_dummies(data_r['treatments'])
just_dummies.head()
#get_dummies() function converts categorical variables into dummy variables. this replaces category values with 0 or 1.

,A,B,C
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False


we will drop column C as we need ony two variables

In [29]:
step1 = pd.concat([data_r, just_dummies], axis = 1)
#here we are concatenating the original data and the dummy data along the axis = 1
step1.drop(['treatments','C'], inplace=True, axis =1)
step1.dropna()
step1.A = step1.A.astype(int)
step1.B = step1.B.astype(int)
step1.head()

,index,value,A,B
0,0,58,1,0
1,1,64,1,0
2,2,55,1,0
3,3,66,1,0
4,4,67,1,0


In [30]:
X = sm.add_constant(step1[['A', 'B']])  # Add constant for intercept
Y = step1['value']
result = sm.OLS(Y, X).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.605
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     9.176
Date:                Tue, 25 Mar 2025   Prob (F-statistic):            0.00382
Time:                        21:52:47   Log-Likelihood:                -44.691
No. Observations:                  15   AIC:                             95.38
Df Residuals:                      12   BIC:                             97.51
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         52.0000      2.380     21.844      0.0

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=15 observations were given.
  return hypotest_fun_in(*args, **kwds)


the F values (0.0038) is lower than the p-value (0.05) and hence we can reject the null hypothesis that the treatment has no effect on the outcome.
and also, the individual coeffiecients are statistically significant.